In [ ]:

import h5py
import matplotlib.pyplot as plt
import numpy as np
import argparse
import importlib
import random
import os
import FLAlgorithms
import utils

#for jupyter notebook reload modules
import imp
imp.reload(FLAlgorithms) 
imp.reload(utils) 
from FLAlgorithms.servers.serveravg import FedAvg
from FLAlgorithms.servers.serverpFedMe import pFedMe
from FLAlgorithms.servers.serverperavg import PerAvg
from FLAlgorithms.trainmodel.models import *
from FLAlgorithms.servers.servertrans import pFedTrans, Cluster

from utils.plot_utils import *

import torch

torch.manual_seed(0)


In [ ]:
def props(cls):   
  return [i for i in cls.__dict__.keys() if i[:1] != '_']

%load_ext autoreload
%autoreload 2

In [ ]:
gpu = 0
device = torch.device("cuda:{}".format(gpu) if torch.cuda.is_available() and gpu != -1 else "cpu")

print("---------------Running time:------------",)
# Generate model
model = 'mclr'
dataset = 'Mnist'

model = Mclr_Logistic().to(device), model
algorithm = "pFedMe"
# select algorithm
batch_size = 20
learning_rate = 0.01
personal_learning_rate = 0.01
beta = 2
lamda = 15
num_glob_iters = 800
local_epochs = 20
optimizer = "SGD"
numusers = 5
K = 5
server = pFedTrans(device, dataset, algorithm, model, batch_size, learning_rate,beta, lamda, num_glob_iters, local_epochs, optimizer, numusers, K, personal_learning_rate, 5)
        

In [ ]:
server.emb_layer.to(device)
for user in server.users:
    user.train(10)
    user.net_values = [*user.model.state_dict().values()]
    user.per_values = user.net_values[-2:]
    user.base_values = user.net_values[:-2]
    value_vec = nn.utils.parameters_to_vector(user.per_values).clone()
    user.emb_vec = server.emb_layer(value_vec)

In [ ]:

import torch
import numpy as np
from kmeans_pytorch import kmeans

In [ ]:
server.evaluate()

In [ ]:
for user in server.users:
    user.train(server.local_epochs)
    user.emb(server.emb_layer)

server.form_cluster()

In [ ]:

for cluster in server.clusters:
    cluster.avg_update_model()
    value_vec = nn.utils.parameters_to_vector(cluster.per_values).clone()
    cluster.emb_vec = server.emb_layer(value_vec)

In [127]:
cluster_emb_list = [cluster.emb_vec.data.clone().reshape(1, -1) for cluster in server.clusters]
x = torch.cat(cluster_emb_list, dim=0).unsqueeze(1)
weights = server.attn_model(x)

In [143]:
for cluster in server.clusters:
    server.intra_cluster_agg(cluster)

d:\Projects\pFedMe-trans\FLAlgorithms\servers\servertrans.py:136: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if glob_iter is not 0:


RuntimeError: Only Tensors created explicitly by the user (graph leaves) support the deepcopy protocol at the moment

In [ ]:

for cluster in server.clusters:
    server.intra_cluster_agg(cluster)

In [ ]:
model = server.clusters[0].model
model_list = []
for param in model.parameters():
    param.grad = param.data
print(value)

In [ ]:
emb_dim = 128
cluster_model_list = [cluster.model.data.clone().reshape(1, -1) for cluster in server.clusters]
attn_dim = 128
x = torch.randn((10, 1, 128)).to(device)
print(x.size())
l1 = nn.Linear(emb_dim, attn_dim).to(device)
l2 = nn.Linear(emb_dim, attn_dim).to(device)

a = l1(x)
b = l2(x)
c = torch.ones_like(a)
output, weight = attn(a, b, c)

print(weight)
torch.einsum('')

In [ ]:
inter_query = inter_query_weight(inter_attn_input)
inter_key = inter_key_weight(inter_attn_input)
inter_value = torch.ones_like(inter_query).to(device)
#inter_key = inter_key_weight(inter_attn_input)
#inter_value = inter_value_weight(inter_query)
attn_output, attn_weight = attn(inter_query, inter_key, inter_value)

In [ ]:
print(attn_weight)

In [ ]:
print(weight)

In [ ]:
z = torch.ones((10, 1, 128)).to(device)
output, weight = attn(x, y, z)
print(weight)

In [ ]:
z = torch.ones((10, 1, 128)).to(device)
attn_output, attn_weight = attn(x, y, z)
print(attn_output)
print(attn_weight)

In [ ]:
torch.rand

In [ ]:
print(dot)

In [ ]:
gpu = 0
device = torch.device("cuda:{}".format(gpu) if torch.cuda.is_available() and gpu != -1 else "cpu")

for i in range(5):
    print("---------------Running time:------------",i)
    # Generate model
    model = 'mclr'
    dataset = 'Mnist'

    model = Mclr_Logistic().to(device), model
    algorithm = "pFedMe"
    # select algorithm
    batch_size = 20
    learning_rate = 0.01
    personal_learning_rate = 0.01
    beta = 2
    lamda = 15
    num_glob_iters = 800
    local_epochs = 20
    optimizer = "SGD"
    numusers = 5
    K = 5
    if(algorithm == "FedAvg"):
        server = FedAvg(device, dataset, algorithm, model, batch_size, learning_rate, beta, lamda, num_glob_iters, local_epochs, optimizer, numusers, i)
        
    if(algorithm == "pFedMe"):
        server = pFedMe(device, dataset, algorithm, model, batch_size, learning_rate, beta, lamda, num_glob_iters, local_epochs, optimizer, numusers, K, personal_learning_rate, i)

    if(algorithm == "PerAvg"):
        server = PerAvg(device, dataset, algorithm, model, batch_size, learning_rate, beta, lamda, num_glob_iters, local_epochs, optimizer, numusers, i)

    server.train()
    server.test()


In [ ]:
import torch
import numpy as np
from kmeans_pytorch import kmeans

# data
data_size, dims, num_clusters = 1000, 100, 3
x = np.random.randn(data_size, dims) / 6
x = torch.from_numpy(x)

# kmeans
cluster_ids_x, cluster_centers = kmeans(
    X=x, num_clusters=num_clusters, distance='euclidean', device=torch.device('cuda:0')
)

In [ ]:

print(cluster_ids_x, cluster_centers.size())

In [ ]:
times = 5

average_data(num_users=numusers, loc_ep1=local_epochs, Numb_Glob_Iters=num_glob_iters, lamb=lamda,learning_rate=learning_rate, beta = beta, algorithms="pFedMe_p", batch_size=batch_size, dataset=dataset, k = K, personal_learning_rate = personal_learning_rate,times = times)
average_data(num_users=numusers, loc_ep1=local_epochs, Numb_Glob_Iters=num_glob_iters, lamb=lamda,learning_rate=learning_rate, beta = beta, algorithms=algorithm, batch_size=batch_size, dataset=dataset, k = K, personal_learning_rate = personal_learning_rate,times = times)

In [ ]:

user = server.users[0]
model = user.model
net_values = [*model.state_dict().values()]
print(net_values)

In [ ]:
print(server.beta)

In [ ]:

for x, y in server.users[0].testloaderfull:
    x = x.to(user.device)
    print(x.size())
    

In [ ]:

# Average data 
if(algorithm == "PerAvg"):
    algorithm == "PerAvg_p"
if(algorithm == "pFedMe"):
    average_data(num_users=numusers, loc_ep1=local_epochs, Numb_Glob_Iters=num_glob_iters, lamb=lamda,learning_rate=learning_rate, beta = beta, algorithms="pFedMe_p", batch_size=batch_size, dataset=dataset, k = K, personal_learning_rate = personal_learning_rate,times = times)
average_data(num_users=numusers, loc_ep1=local_epochs, Numb_Glob_Iters=num_glob_iters, lamb=lamda,learning_rate=learning_rate, beta = beta, algorithms=algorithm, batch_size=batch_size, dataset=dataset, k = K, personal_learning_rate = personal_learning_rate,times = times)

